In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import gsw

from scipy.integrate import solve_ivp

In [2]:
ds = xr.open_dataset('data/MBARI_StationM_ADCP_201711_201811.nc')

In [3]:
lat = 35+8.4585/60
bini = 15

Smooth ADCP time series
* 30-min running median to remove spikes
* 60-min running mean to smooth

In [4]:
ds['Eas_sm'] = ds['Eas'].rolling(time=6,center=True).median().rolling(time=12,center=True).mean()
ds['Nor_sm'] = ds['Nor'].rolling(time=6,center=True).median().rolling(time=12,center=True).mean()

In [5]:
plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Eas_sm'][:,bini]);
plt.plot(ds['time'],ds['Eas'][:,bini],'--');
plt.plot(ds['time'],ds['Eas_sm'][:,1],'-');
#plt.plot(ds['time'],ds['Eas'][:,1]);
plt.ylabel('m/s')
plt.title('eastward velocity')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'eastward velocity')

In [6]:
plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Nor_sm'][:,bini]);
plt.legend(['Bin 15','Bin 1'])
plt.ylabel('m/s')
plt.title('northward velocity')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'northward velocity')

### Test with limited obs


In [7]:
zf = ds['binheight'][bini] # upper boundary height
zobs = np.array(ds['binheight'][0:(bini+1)])

In [8]:
zo = 0.03 # roughness length
i = 1j
kappa = 0.41 # Von Karman constant
f = gsw.f(lat)

z = np.logspace(np.log10(zo),np.log10(zf),201)
dz = np.diff(z)

In [9]:
gi = np.isfinite(ds['Eas_sm'][:,0])
tobs = np.array((ds['time'][gi] - ds['time'][gi][0])/np.timedelta64(1,'s')) # array of times in seconds
deltat = tobs[1]-tobs[0]

In [10]:
wobs = np.array(ds['Eas_sm'][gi,0:(bini+1)]+i*ds['Nor_sm'][gi,0:(bini+1)])

w0_obs = np.interp(z,zobs[1:bini],wobs[0,1:bini])[1:-1]
w0 = np.concatenate([np.real(w0_obs),
                           np.imag(w0_obs)])

dwobsdt = np.gradient(wobs,deltat,axis=0)

In [11]:
def dwdt_bbl(t, w_in):
    # Note that w_in is real and complex parts concatenates
    # need to combine into complex numbers
    # Scipy docs say that ode solvers RK45 and BDF can be used in complex plane
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.BDF.html#scipy.integrate.BDF
    # but perhaps this does not work for *arrays* of complex numbers?
    # Test this and check Numpy Github issues for similar problems
    
    
    N = int(len(w_in)/2)
    w = w_in[:N]+i*w_in[N:]
    
    winf = np.interp(t,tobs,wobs[:,bini])
    dwinfdt = np.interp(t,tobs,dwobsdt[:,bini])

    wall = np.concatenate([np.atleast_1d(0+i*0),w,np.atleast_1d(winf)])
    
    ustar = kappa*zo*np.abs(w[1])/dz[0]
    coef1 = 2*kappa*ustar/(dz[:-1]+dz[1:])
    
    st =  (coef1*(z[1:-1]/dz[1:])*wall[2:]
         - coef1*(z[1:-1]/dz[1:]+z[:-2]/dz[:-1])*wall[1:-1]
         + coef1*(z[:-2]/dz[:-1])*wall[:-2])
    
    F = dwinfdt + i*f*winf + st[-1]

    
    dwalldt = np.nan*np.ones(np.shape(wall))+i*np.nan*np.ones(np.shape(wall))
    dwalldt[1:-1] = (F +
                     coef1*(z[1:-1]/dz[1:])*wall[2:] +
                     (-i*f - coef1*(z[1:-1]/dz[1:]+z[:-2]/dz[:-1]))*wall[1:-1] +
                     coef1*(z[:-2]/dz[:-1])*wall[:-2])
    
    dwdt = dwalldt[1:-1]
    
    dwdt_out = np.concatenate([np.real(dwdt),np.imag(dwdt)])
    
    return dwdt_out

In [12]:
#%%timeit

#tf = tobs[-1]
tf = 3600


tic = np.datetime64('now')
print(tic)
a = solve_ivp(dwdt_bbl, t_span=[tobs[0],tf], y0 = w0, t_eval = tobs[tobs<=tf], method='BDF')
toc = np.datetime64('now')
print(toc)
print(toc-tic)
#a = solve_ivp(dwdt_bbl, t_span=[tobs[0],tf], y0 = w0, method='BDF')

2020-01-13T08:22:45
2020-01-13T08:22:52
7 seconds


In [13]:
N = int(np.shape(a.y)[0]/2)

t_sol = a.t
w_sol = a.y[:N,:]+i*a.y[N:,:]

In [14]:
plt.figure()
plt.subplot(211)
plt.plot(t_sol,np.real(w_sol).T,color='gray')
plt.plot(tobs[0:12],np.real(wobs[0:12,bini]))

plt.subplot(212)
plt.plot(t_sol,np.imag(w_sol).T,color='gray')
plt.plot(tobs[0:12],np.imag(wobs[0:12,bini]))

FigureCanvasNbAgg()

In [15]:
np.diff(a.t[0:100])

array([300., 300., 300., 300., 300., 300., 300., 300., 300., 300., 300.,
       300.])

### Test with steady forcing

In [30]:
winf_steady = 0.08+i*0
w0_steady = np.concatenate([np.real(winf_steady)*np.ones(N),
                           np.imag(winf_steady)*np.ones(N)])
zo = 0.001

def dwdt_bbl_steady(t, w_in):
    N = int(len(w_in)/2)
    w = w_in[:N]+i*w_in[N:]
    
    winf = winf_steady
    dwinfdt = 0+i*0

    
    ustar = kappa*zo*np.abs(w[1])/dz[0]
    coef1 = 2*kappa*ustar/(dz[:-1]+dz[1:])

    wall = np.concatenate([np.atleast_1d(0+i*0),w,np.atleast_1d(winf)])
    
    st =  (coef1*(z[1:-1]/dz[1:])*wall[2:]
     - coef1*(z[1:-1]/dz[1:]+z[:-2]/dz[:-1])*wall[1:-1]
     + coef1*(z[:-2]/dz[:-1])*wall[:-2])
    
    F = dwinfdt + i*f*winf - st[-1]

    dwalldt = np.nan*np.ones(np.shape(wall))+i*np.nan*np.ones(np.shape(wall))
    dwalldt[1:-1] = (F +
                     coef1*(z[1:-1]/dz[1:])*wall[2:] +
                     (-i*f - coef1*(z[1:-1]/dz[1:]+z[:-2]/dz[:-1]))*wall[1:-1] +
                     coef1*(z[:-2]/dz[:-1])*wall[:-2])
    
    dwdt = dwalldt[1:-1]
    #print(dwdt)
    
    dwdt_out = np.concatenate([np.real(dwdt),np.imag(dwdt)])
    
    return dwdt_out

In [31]:
modsteady = solve_ivp(dwdt_bbl_steady, t_span=[0,2*86400], 
                      y0 = w0_steady, method='BDF')

In [32]:
t_sol_steady = modsteady.t
w_sol_steady = modsteady.y[:N,:]+i*modsteady.y[N:,:]

In [33]:
plt.figure()
plt.subplot(121)
plt.plot(np.real(w_sol_steady[:,-1]),z[1:-1]);
plt.plot(np.imag(w_sol_steady[:,-1]),z[1:-1]);
#plt.plot(tobs[0:12*24],wobs[0:12*24,bini])

FigureCanvasNbAgg()

In [34]:
ustar_steady = kappa*zo*np.abs(w_sol_steady[1,-1])/dz[0]

In [35]:
ustar_steady

0.0005632123088565189

In [36]:
dz

array([0.00096747, 0.00099867, 0.00103087, 0.00106412, 0.00109844,
       0.00113386, 0.00117042, 0.00120817, 0.00124713, 0.00128735,
       0.00132887, 0.00137172, 0.00141596, 0.00146162, 0.00150876,
       0.00155741, 0.00160764, 0.00165948, 0.001713  , 0.00176824,
       0.00182526, 0.00188413, 0.00194489, 0.00200761, 0.00207235,
       0.00213918, 0.00220817, 0.00227938, 0.00235289, 0.00242877,
       0.00250709, 0.00258794, 0.0026714 , 0.00275755, 0.00284648,
       0.00293828, 0.00303303, 0.00313084, 0.00323181, 0.00333603,
       0.00344362, 0.00355467, 0.0036693 , 0.00378763, 0.00390978,
       0.00403587, 0.00416602, 0.00430037, 0.00443905, 0.00458221,
       0.00472998, 0.00488252, 0.00503997, 0.0052025 , 0.00537028,
       0.00554347, 0.00572224, 0.00590677, 0.00609726, 0.00629389,
       0.00649686, 0.00670638, 0.00692265, 0.0071459 , 0.00737635,
       0.00761423, 0.00785978, 0.00811325, 0.00837489, 0.00864497,
       0.00892376, 0.00921155, 0.00950861, 0.00981525, 0.01013